# Drosophilia Data

This notebook will compute mutual information on the drosophilia data.


In [2]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [3]:
data = pd.read_csv("data/vdf.csv")
print(data)

                                                     v type  claw  \
0     Thermo KC A_Right - Mushroom Body  Left #4229950   KC   2.0   
1                                        KC12 #4230061   KC   1.0   
2                                        KC10 #4227304   KC   1.0   
3                                        KC42 #8066542   KC   1.0   
4                                        KC16 #4415241   KC   2.0   
5                                      KC100 #11813287   KC   2.0   
6                                LON KC right #8068173   KC   4.0   
7                                        KC21 #5051342   KC   1.0   
8                                         KC2 #3347708   KC   1.0   
9                                        KC13 #4230739   KC   1.0   
10                                       KC90 #9796812   KC   1.0   
11         Thermo KC B_right - mbi2 - unknown #3664102   KC   4.0   
12                                      KC96 #10056280   KC   2.0   
13                                

In [4]:
data.type = pd.factorize(data.type)[0]
data.age =pd.factorize(data.age)[0]
data.v =pd.factorize(data.v)[0]
data = data.fillna(0)

In [5]:
print(data)

       v  type  claw         dist  age  cluster
0      0     0   2.0   337.638010    0        1
1      1     0   1.0     0.000000    0        1
2      2     0   1.0  1291.203570    0        1
3      3     0   1.0   252.633470    0        1
4      4     0   2.0  1822.295250    0        1
5      5     0   2.0  1031.646020    0        1
6      6     0   4.0  3928.179840    0        1
7      7     0   1.0  5966.114650    0        1
8      8     0   1.0  7994.829270    0        1
9      9     0   1.0   946.986460    0        1
10    10     0   1.0  4249.320010    0        1
11    11     0   4.0     0.000000    0        1
12    12     0   2.0  4249.320010    0        1
13    13     0   1.0  3042.089030    0        1
14    14     0   2.0  8278.148080    0        1
15    15     0   1.0  4249.320010    0        1
16    16     0   2.0  2497.301090    0        1
17    17     0   1.0  5966.114650    0        1
18    18     0   1.0  2721.851950    0        1
19    19     0   1.0  7878.687810    0  

In [6]:
y = data["type"].tolist()

In [7]:
y[:5]

[0, 0, 0, 0, 0]

In [8]:
data = data.drop(columns=["type"])

In [9]:
X = data.as_matrix()

/usr/local/Cellar/jupyter/1.0.0_5/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [10]:
print(X.shape)

(213, 5)


In [11]:
import sys
sys.path.insert(0, "code")
import entropy_estimators as ee
import mixed

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
print(y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [16]:
from sklearn.ensemble.forest import _generate_unsampled_indices
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import math


def rf_entropy_estimate_bagging(X, y, n_estimators = 200, max_samples = .32, bootstrap = True, depth = 30, min_samples_leaf = 1, max_features = 1.):
    model = BaggingClassifier(DecisionTreeClassifier(max_depth = depth, min_samples_leaf = min_samples_leaf, max_features = math.ceil(int(math.sqrt(X.shape[1])))), 
                              n_estimators = n_estimators, 
                              max_samples= max_samples, 
                              bootstrap = bootstrap)
    model.fit(X, y)
    class_counts = np.zeros((X.shape[0], model.n_classes_))
    for tree in model:
        # get out of bag indicies
        unsampled_indices = _generate_unsampled_indices(tree.random_state, len(X))
        
        total_unsampled = len(unsampled_indices)
        np.random.shuffle(unsampled_indices)
        prob_indices, eval_indices = unsampled_indices[:total_unsampled//2], unsampled_indices[total_unsampled//2:]
        # get all node counts
        node_counts = tree.tree_.n_node_samples
        # get probs for eval samples
        posterior_class_counts = np.zeros((len(node_counts), model.n_classes_))
        for prob_index in prob_indices:
            posterior_class_counts[tree.apply(X[prob_index].reshape(1, -1)).item(), y[prob_index]] += 1
        row_sums = posterior_class_counts.sum(axis=1)
        row_sums[row_sums == 0] = 1
        class_probs = (posterior_class_counts/row_sums[:, None])
        
        where_0 = np.argwhere(class_probs == 0)
        for elem in where_0:
            class_probs[elem[0], elem[1]] = 1/(model.n_classes_*row_sums[elem[0], None])
        where_1 = np.argwhere(class_probs == 1)
        for elem in where_1:
            class_probs[elem[0], elem[1]] = 1 - (model.n_classes_ - 1)/(model.n_classes_*row_sums[elem[0], None])
        
        class_probs.tolist()
        partition_counts = np.asarray([node_counts[x] for x in tree.apply(X[eval_indices])])
        # get probability for out of bag samples
        eval_class_probs = [class_probs[x] for x in tree.apply(X[eval_indices])]
        eval_class_probs = np.array(eval_class_probs)
        # find total elements for out of bag samples
        elems = np.multiply(eval_class_probs, partition_counts[:, np.newaxis])
        # store counts for each x (repeat fhis for each tree)
        class_counts[eval_indices] += elems
    # calculate p(y|X = x) for all x's
    probs = class_counts/class_counts.sum(axis = 1, keepdims = True)
    entropies = -np.sum(np.log(probs)*probs, axis = 1)
    # convert nan to 0
    entropies = np.nan_to_num(entropies)
    return np.mean(entropies)

np.warnings.filterwarnings('ignore')


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [17]:
ce = rf_entropy_estimate_bagging(X, y)

In [23]:
mixed.Mixed_KSG(X, np.array(y).reshape(-1, 1))/entropy_y

0.04683911099354165

In [19]:
import numpy as np
from scipy.stats import entropy
from math import log, e
import pandas as pd

In [20]:


import numpy as np
from scipy.stats import entropy
from math import log, e
import pandas as pd

def entropy1(labels, base=None):
    values,counts = np.unique(labels, return_counts=True)
    return entropy(counts, base=base)

entropy_y = entropy1(y)
print(entropy_y)

1.2151839186517945


In [22]:
(entropy_y - ce)/entropy_y

0.648557756924189

In [15]:
ee.mi(X, np.array(y).reshape(-1, 1))

0.2434463844308495